In [36]:
# Import library-library
import os
import glob

# Data Preparation and Preprocessing
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords = set(NLTK_StopWords)

# Word Embedding
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

from sklearn.metrics.pairwise import cosine_similarity

# Input and Expansion Query
from textblob import TextBlob
from nltk.tokenize import wordpunct_tokenize
#from googletrans import Translator

#Split to train and test
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/syubbanfakhriya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
excluded_words = ["tempat", "waktu"]
NLTK_StopWords.update(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali", "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom"])

In [38]:
def preprocessing(berita):
    s = berita.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    s = s.replace(' o ', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    s = re.sub(r'[0-9]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    T = [t for t in tokens if ((t in excluded_words) or (t not in NLTK_StopWords))]
    return T

In [53]:
df =pd.read_csv("news.csv")
df.info()
# df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1790 non-null   object
 1   date         1790 non-null   object
 2   description  1790 non-null   object
 3   source       1790 non-null   object
dtypes: object(4)
memory usage: 56.1+ KB


In [40]:
column = ['Title', 'Date', 'Description', 'Source']
df_total = pd.DataFrame()

df =pd.read_csv("news.csv")
isi=df['description']
df = df.dropna(subset=['description'], axis=0)
for x in range(0, df.shape[0]):
    text = preprocessing (df.iloc[x, -2])
    df.iloc[x, -2] = (' '.join(text)) 

In [41]:
df.head()
df_train, df_test=train_test_split(df,test_size=0.1)
df_train.head()

,title,date,description,source
813,\n Ini 9 Gempa Dahsyat Pernah Terjadi d...,"Senin, 12 Apr 2021 11:22 WIB",wilayah selatan jawa timur dikenal kawasan sei...,www.detik.com
1654,\n Shopee Ajak Pengguna Donasi ke Korba...,"Jumat, 22 Jan 2021 17:39 WIB",pandemi covid dunia indonesia selesai wabah pe...,www.detik.com
1038,\n 2 Penjaga Alat Berat di Aceh Tewas T...,"Kamis, 02 Des 2021 15:24 WIB",penjaga alat berat bireuen aceh tewas tertimbu...,www.detik.com
1658,\n Distribusi Bantuan untuk Korban Banj...,"Sabtu, 23 Jan 2021 12:37 WIB",wilayah jember jawa timur mengalami bencana ba...,www.detik.com
99,"\n Di SOFO 2020, KLHK Sebut Deforestasi...","Jumat, 22 Mei 2020 21:48 WIB",menteri lingkungan hidup kehutanan lhk siti nu...,www.detik.com


In [42]:
desc_text_train = []
desc_text_test = []

for x in range(0, df_train.shape[0]):
  desc_text_train.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test.append(df_test.iloc[x, -2])

In [43]:
print(df_test.info())
print("Panjang Dokumen Test : ", len(desc_text_test))

print ('-'*90)

print(df_train.info())
print("Panjang Dokumen Train : ", len(desc_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 1537 to 592
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        179 non-null    object
 1   date         179 non-null    object
 2   description  179 non-null    object
 3   source       179 non-null    object
dtypes: object(4)
memory usage: 7.0+ KB
None
Panjang Dokumen Test :  179
------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1611 entries, 813 to 931
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1611 non-null   object
 1   date         1611 non-null   object
 2   description  1611 non-null   object
 3   source       1611 non-null   object
dtypes: object(4)
memory usage: 62.9+ KB
None
Panjang Dokumen Train :  1611


In [44]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test, "corpus/model/desc_text_test.pkl")
joblib.dump(desc_text_train, "corpus/model/desc_text_train.pkl")

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('corpus/dataset/df_total.csv',index=False )
df_test.to_csv('corpus/dataset/df_test.csv', index=False)
df_train.to_csv('corpus/dataset/df_train.csv', index=False)


In [45]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_test_matrix = tfidf_vectorizer.fit_transform(desc_text_test)
tfidf_train_matrix = tfidf_vectorizer.fit_transform(desc_text_train)

print('tfidf_test_matrix')
print(tfidf_test_matrix.shape)
print('tfidf_train_matrix')
print(tfidf_train_matrix.shape)

tfidf_test_matrix
(179, 4718)
tfidf_train_matrix
(1611, 15903)


In [46]:
joblib.dump(tfidf_vectorizer, "corpus/vectorizer/vectorizer.pkl")
joblib.dump(tfidf_test_matrix, "corpus/matrix/tfidf_test.pkl")
joblib.dump(tfidf_train_matrix, "corpus/matrix/tfidf_train.pkl")

['corpus/matrix/tfidf_train.pkl']

### Pencarian 

In [47]:
df_total = pd.read_csv('corpus/dataset/df_total.csv')
df_total = df_total[pd.notnull(df_total['description'])]
print(df_total.info())
print ('-'*90)

document_text_train= joblib.load('corpus/model/desc_text_train.pkl')
document_text_test= joblib.load('corpus/model/desc_text_test.pkl')
print(len(document_text_test))
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1790 entries, 0 to 1789
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1790 non-null   object
 1   date         1790 non-null   object
 2   description  1790 non-null   object
 3   source       1790 non-null   object
dtypes: object(4)
memory usage: 69.9+ KB
None
------------------------------------------------------------------------------------------
179
1611


In [52]:
kueri='gempa'
kueri=preprocessing(kueri)
kueri= [" ".join (kueri)]
print (kueri)

tfidf_matrix =joblib.load( "corpus/matrix/tfidf_train.pkl" )
tfidf_vectorizer = joblib.load( "corpus/vectorizer/vectorizer.pkl" ) 
query_vec= tfidf_vectorizer.transform(kueri)
results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#print (results)
print("\n======================\n")
print("Top 10 most similar documents in corpus:")

j = 1

for i in results.argsort()[-10:][::-1]:
    print(j)
    print("No ID Dokumen  : ", i)
    print("Tanggal        : ", df_total.iloc[i,1])
    print("Isi berita     : ", df_total.iloc[i,2])
    print("(Score: %.4f) " % results[i])
    j += 1

['gempa']


Top 10 most similar documents in corpus:
1
No ID Dokumen  :  257
Tanggal        :  Rabu, 11 Agu 2021 02:58 WIB
Isi berita     :  melanda dilansir foto foto tersebar media sosial menampilkan kepulan asap api membumbung pepohonan hutan terbakar wilayah kabylie sebelah timur kota aljir presiden aljazair abdelmadjid tebboune berbelasungkawa tentara tewas tentara tentara menyelamatkan pria wanita anak anak sedih kematian tentara berhasil menyelamatkan warga kobaran api pegunungan bejaia tizi ouzou tulis tebboune via twitternya dilansir salah radio pemerintah aljazair mengabarkan pembakar ditangkap distrik utara medea annaba kebakaran negara aljazair ahli meteorologi suhu aljazair mencapai derajat celcius selasa aljazair salah sekian negara dilanda kebakaran pekan aljazair yunani turki siprus amerika serikat barat
(Score: 0.6479) 
2
No ID Dokumen  :  1172
Tanggal        :  Jumat, 25 Feb 2022 13:42 WIB
Isi berita     :  magnitudo m mengguncang pasaman barat sumatera barat kepala b